<a href="https://colab.research.google.com/github/IgnacioCourt/Proyecto_Final/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Final - Grupo 13


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas
import csv
import time
from datetime import date
from datetime import datetime
from math import ceil
from clases import medicamento


In [ ]:
class medicamento:
    def __init__(self,principio_a,farmacia,descripcion,precio,preciouf):
        self.principio_activo=principio_a
        self.farmacia=farmacia
        self.descripcion=descripcion
        self.precio=precio
        self.preciouf=preciouf

In [ ]:


def to_csv(lista):
    name="parametros.csv"
    archivo2 = open(name,"w")

    name="medicamentos.csv"
    with open(name,"w",newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerows(lista)

def UF():
    url="https://www.bcentral.cl/inicio"
    page= requests.get(url)
    soup= BeautifulSoup(page.content,"html.parser")
    uf=soup.find("p",class_="basic-text fs-2 f-opensans-bold text-center c-blue-nb-2")
    fecha=soup.find("p",class_="basic-text f-opensans-semibold c-beige-nb-1 fs-1 mb-1 text-center text-lg-left")
    uf=uf.text[1:]
    uftemp=""
    for i in uf:
        if i==".":
            continue
        if i==",":
            uftemp+="."
            continue
        uftemp+=i
    uf=uftemp
    print(uf)

    print(fecha.text)
    
    lista=[[fecha.text,uf]]
    #Poner los datos en el CSV
    return uf

def principios_activos():
    lista_med=[]
    with open('princios_activos.txt', 'r', encoding='utf8') as fichero:
        lista = fichero.readlines()
    for i in lista:
        if i[-1:]=="\n":
            lista_med.append(str(i[:-1]))
        else:
            lista_med.append(i)
    return lista_med

def ahumada(lista,uf,general):
    count=0
    precios=[]
    descripciones=[]
    nombres=[]
    medicamentos=[]
    preciosufs=[]
    for med in lista:
    
        url="https://www.farmaciasahumada.cl/catalogsearch/result/index/?p=10000&q="+med
        page= requests.get(url)
        soup= BeautifulSoup(page.content,"html.parser")
        #left-search-count
        cant=soup.find("span",class_="left-search-count")

        cantidad=""
        for i in cant.text:
           
            if i=="P" or i=="p":
                break
            cantidad+=i
        cantidad=int(cantidad.strip())
        if cantidad!=0:
            paginas=ceil(cantidad/12)
        else:
            paginas=0
        #print(paginas)
        #aqui hay que sacar la cantidad de página
        #paginas=2
        for i in range(1,paginas+1):
            url="https://www.farmaciasahumada.cl/catalogsearch/result/index/?p="+str(i)+"&q="+med
            page= requests.get(url)
            soup= BeautifulSoup(page.content,"html.parser")
            nombre=soup.findAll("p",class_="product-brand-name truncate")
            precio=soup.findAll("span",class_="price-container price-final_price tax weee")
            farmacia="Ahumada"
            descripcion=soup.findAll("strong",class_="product name product-item-name truncate")

          
            for a in descripcion:
                descripciones.append(a.text.strip())
            for nom in nombre:
                nombres.append(nom.text.strip())
            
            
            # for nom in nombre:
                
            #     print(nom.text)
           
            
            for pre in precio:
                prec=""
                es=0
                for i in pre.text:
                    if i=="$":
                        es=1 
                        continue  
                    if es==1:
                        if i==".":
                            continue
                        if i=="":
                            break
                        prec+=str(i)
                        
                           
                prec=int(prec.strip())
                precios.append(prec)
                preciosufs.append(round(prec/uf,2))   
            
    # count+=1
    
    # print(len(nombres),"n----------------")
    # print(len(descripciones),"d----------------")
    # print(len(precios),"p----------------")

    for i in range(len(nombres)):
        temp=[nombres[i],farmacia,descripciones[i],precios[i],preciosufs[i]]
        general.append(temp)
        medi=medicamento(nombres[i],farmacia,descripciones[i],precios[i],preciosufs[i])
        medicamentos.append(medi)

        # if count>0:
        #     break
    
    return general, medicamentos


def farmachile(lista, uf, general):
    count=0
    precios=[]
    descripciones=[]
    nombres=[]
    medicamentos=[]
    preciosufs=[]
    farmacia="farmacias chile"
    for med in lista:

        url= "https://www.farmaciaschilespa.cl/?s="+med
        page= requests.get(url)
        soup= BeautifulSoup(page.content, "html.parser")
        cant= soup.find("h1", class_="title")
        cantidad=""
        for i in cant.text:
           
            if i=="r" or i=="R":
                break
            cantidad+=i
        cantidad=int(cantidad.strip())
        if cantidad!=0:
            paginas=ceil(cantidad/4)
        else:
            paginas=0
       
      
        #paginas=1
        for i in range(1, paginas+1):
            url="https://www.farmaciaschilespa.cl/page/"+str(i)+"/?s="+med
            page= requests.get(url)
            soup= BeautifulSoup(page.content, "html.parser")
            nombre= soup.findAll("div", class_="post-title")
            precio= soup.findAll("div", class_="post-product-price")
            farmacia= "FarmaciasChile"
            descripcion= soup.findAll("div", class_="post-excerpt")
            
            for a in descripcion:
                descripciones.append(a.text.strip())
                #print(a.text.strip())
            for nom in nombre:
                nombres.append(nom.text.strip())
                #print(nom.text.strip())
            for pre in precio:
                prec=""
                es=0
                for i in pre.text:
                    if i=="C":
                        
                        break
                    prec+=str(i)
                prec=int(prec.strip())
                precios.append(prec)
                preciosufs.append(round(prec/uf, 2))



    for i in range(len(nombres)):
      temp=[nombres[i], farmacia, descripciones[i], precios[i], preciosufs[i]]
      general.append(temp)
      medi=medicamento(nombres[i], farmacia, descripciones[i], precios[i], preciosufs[i])
      medicamentos.append(medi)
    
    return general,medicamentos

def main():
    total_medicamentos=[]
    medicamentos_clases=[]
    uf=float(UF())
    lista_medicamentos=principios_activos()
    total_medicamentos,medicamentos_clases=ahumada(lista_medicamentos,uf,total_medicamentos)
    total_medicamentos,medicamentos_clases=farmachile(lista_medicamentos,uf,total_medicamentos)
    #otrafarmacia(lista_medicamentos,uf,total_medicamentos)
    to_csv(total_medicamentos)



if __name__== "__main__":
    main()

33047.39
27 de Junio de 2022
